In [4]:
%pip install langchain langchain-community langchain_ollama langchain_pinecone

  Using cached langchain_pinecone-0.2.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached pinecone-5.4.2-py3-none-any.whl.metadata (19 kB)
  Using cached pytest_asyncio-0.25.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached pytest_socket-0.7.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached syrupy-4.8.1-py3-none-any.whl.metadata (36 kB)
  Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain_pinecone-0.2.2-py3-none-any.whl (11 kB)
Using cached pinecone-5.4.2-py3-none-any.whl (427 kB)
Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl (87 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Using cached pytest_asyncio-0.25.2-py3-none-any.whl (19 kB)
Using cached pytest_socket-0.7.0-py3-none-any.whl (6.8 kB)
Using cached syrupy-4.8.1-py3-none-any.whl (50 kB)
  Attempting uninstall: aiohttp
    Found existing installation: 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-ollama 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
llama-index-embeddings-ollama 0.3.1 requires ollama<0.4.0,>=0.3.1, but you have ollama 0.4.5 which is incompatible.
llama-index-llms-ollama 0.3.6 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
llama-index-readers-google 0.4.3 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
llama-index-vector-stores-chroma 0.3.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
open-webui 0.3.35 requires aiohttp==3.10.8, but you have aiohttp 3.10.11 which is incompatible.
open-webui 0.3.35 requires boto3==1.35.0, but you have boto3 1.

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

# Configure Ollama LLM
ollama_llm = OllamaLLM(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest",  # Correct parameter name is `model`
    base_url="http://localhost:11434",  # Base URL for the Ollama service
)




In [7]:
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [16]:
import time

index_name = "sample"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [11]:
pc.describe_index(index_name)

{
    "name": "sample",
    "dimension": 384,
    "metric": "cosine",
    "host": "sample-zhmtpmp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [12]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]



In [17]:
vectorstore = PineconeVectorStore.from_documents(
    docs, index_name=index_name, embedding=ollama_embedding
)

In [18]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [19]:
result = retriever.invoke("dinosaur")
result

(Document(id='b8b86bd1-df85-4efa-ab88-cf228170dee7', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.701512575}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='5e6563b0-3bfa-402b-8ba7-d4e5dd7cd073', metadata={'genre': 'animated', 'year': 1995.0, 'score': 0.541614771}, page_content='Toys come alive and have a blast doing so'),
 Document(id='287dc636-1fbd-4f9d-8b24-6f1ce0224301', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979.0, 'score': 0.420702457}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='ec329a73-1770-4bd3-9e8d-41b4ce196b3b', metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010.0, 'score': 0.417136729}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'))